<a href="https://colab.research.google.com/github/tdurand06/Solar-detection-optuna/blob/main/pv_fault_optuna_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
from sklearn.utils import resample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

drive.mount('/content/drive')

drive_file_path1 = '/content/drive/MyDrive/Solar/dataset_amb.mat'
drive_file_path2 = '/content/drive/MyDrive/Solar/dataset_elec.mat'

amb_data = loadmat(drive_file_path1)
elec_data = loadmat(drive_file_path2)
pd_elec_series=pd.Series(elec_data)
pd_elec_series = pd_elec_series.drop([pd_elec_series.index[0],pd_elec_series.index[1],pd_elec_series.index[2]], axis=0)
df_elec = pd.DataFrame({
    name: values[0] for name, values in pd_elec_series.items()
})

pd_amb_series = pd.Series(amb_data)
pd_amb_series = pd_amb_series.drop([pd_amb_series.index[0], pd_amb_series.index[1], pd_amb_series.index[2]])
def process_values(values):
    # If it's like irr or pvt ([[1.3729, 1.3604, ...]])
    if len(values[0]) > 1:
        return values[0]
    # If it's like f_nv ([[0], [0], [0], ...])
    else:
        return [val[0] for val in values]

df_amb = pd.DataFrame({
    name: process_values(values)
    for name, values in pd_amb_series.items()
})
X = pd.DataFrame({
    'idc1': df_elec['idc1'],
    'idc2': df_elec['idc2'],
    'vdc1': df_elec['vdc1'],
    'vdc2': df_elec['vdc2'],
    'irr': df_amb['irr'],
    'pvt': df_amb['pvt']
})
y = pd.DataFrame({
    'f_nv': df_amb['f_nv']
})

print(X.shape)
print(y.shape)

class PVFaultDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        if isinstance(y, pd.DataFrame):
            y = y.values
        self.y = torch.tensor(y, dtype=torch.int64).squeeze()


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def preprocess_data_subsample_three_splits(X, y, test_size=0.15, val_size=0.15):
    class_counts = np.unique(y, return_counts=True)[1]
    min_count = np.min(class_counts)
    X_train_balanced = []
    y_train_balanced = []

    for i in np.unique(y):
        # Use boolean indexing to extract, use values.ravel() to match the dimensions.
        mask = y.values.ravel() == i
        X_class = X[mask]
        y_class = y[mask]
        X_class_resampled, y_class_resampled = resample(X_class, y_class, n_samples=min_count, random_state=18)
        X_train_balanced.append(X_class_resampled)
        y_train_balanced.append(y_class_resampled)
        # transform list back into (1373798, 6) (1373798, 1) respectively
    X_trained_balanced = np.concatenate(X_train_balanced)
    y_trained_balanced = np.concatenate(y_train_balanced)
    #First split
    X_train_val, X_test, y_train_val, y_test = train_test_split(X_trained_balanced, y_trained_balanced, test_size=test_size, random_state=42)
    # Second split: separate validation set
    val_size_adjusted = val_size / (1 - test_size)  # Adjust validation size
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val,
                                                      test_size=val_size_adjusted,
                                                      random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_train_val = scaler.transform(X_train_val)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_dataset = PVFaultDataset(X_train, y_train)
    val_dataset = PVFaultDataset(X_val, y_val)
    train_val_dataset = PVFaultDataset(X_train_val, y_train_val)
    test_dataset = PVFaultDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=32, num_workers=2, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, num_workers=2, shuffle=False)
    train_val_loader = DataLoader(train_val_dataset, batch_size=32, num_workers=2, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, num_workers=2, shuffle=False)

    return train_loader, val_loader, train_val_loader, test_loader, scaler

class PVFaultClassifier(nn.Module):
    def __init__(self, dropout_rate1=None, dropout_rate2=None):
        super(PVFaultClassifier, self).__init__()
        layers = [
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        ]
        if dropout_rate1 is not None:
            layers.append(nn.Dropout(dropout_rate1))
        layers += [
            nn.MaxPool1d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.ReLU()
        ]
        if dropout_rate2 is not None:
            layers.append(nn.Dropout(dropout_rate2))
        layers.append(nn.Linear(64, 5))
        self.model = nn.Sequential(*layers)


    def forward(self, x):
        return self.model(x)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1373798, 6)
(1373798, 1)



# TrainingMetrics Class
 Responsible for tracking, storing, and managing all metrics
during the model training process. This class maintains a history of performance
metrics and provides methods to update and retrieve this information.

Key responsibilities:
- Store training and validation metrics (losses and accuracies)
- Track the best model state based on validation loss
- Store prediction and target data for confusion matrix generation
- Provide methods to retrieve metrics in formats suitable for visualization


In [ ]:


class TrainingMetrics:
    def __init__(self):
        """Initialize metrics storage containers"""
        # Store all metrics in dictionaries for consistency and easy access
        self.metrics = {
            'train_losses': [],       # Training loss history
            'val_losses': [],         # Validation loss history
            'train_accuracies': [],   # Training accuracy history
            'val_accuracies': []      # Validation accuracy history
        }

        # Store data needed for confusion matrix and additional analysis
        self.data = {
            'pred_train': [],         # Predictions on training data
            'target_train': [],       # Ground truth for training data
            'pred_val': [],           # Predictions on validation data
            'target_val': []          # Ground truth for validation data
        }

        # Track best model state and validation loss
        self.best_val_loss = float('inf')
        self.best_model_state = None

    def update(self, train_loss, train_acc, val_loss, val_acc, predictions=None, targets=None):
        """
        Update metrics after an epoch of training

        Args:
            train_loss: The training loss for this epoch
            train_acc: The training accuracy for this epoch
            val_loss: The validation loss for this epoch
            val_acc: The validation accuracy for this epoch
            predictions: Model predictions on validation data (optional)
            targets: True labels for validation data (optional)

        Returns:
            bool: True if this model has the best validation loss so far
        """
        # Update metric history
        self.metrics['train_losses'].append(train_loss)
        self.metrics['val_losses'].append(val_loss)
        self.metrics['train_accuracies'].append(train_acc)
        self.metrics['val_accuracies'].append(val_acc)

        # Store predictions and targets if provided
        if predictions is not None and targets is not None:
            self.data['pred_val'] = predictions
            self.data['target_val'] = targets

        # Check if this is the best model so far
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            return True
        return False

    def update_confusion_matrix_data(self, pred_train, target_train, pred_val, target_val):
        """
        Store all data needed for confusion matrix visualization

        Args:
            pred_train: Predictions on training data
            target_train: Ground truth for training data
            pred_val: Predictions on validation data
            target_val: Ground truth for validation data
        """
        self.data['pred_train'] = pred_train
        self.data['target_train'] = target_train
        self.data['pred_val'] = pred_val
        self.data['target_val'] = target_val

    def get_summary(self):
        """
        Create a summary of training results

        Returns:
            dict: Summary statistics of the training process
        """
        return {
            'best_val_loss': self.best_val_loss,
            'final_train_loss': self.metrics['train_losses'][-1] if self.metrics['train_losses'] else None,
            'final_val_loss': self.metrics['val_losses'][-1] if self.metrics['val_losses'] else None,
            'final_train_acc': self.metrics['train_accuracies'][-1] if self.metrics['train_accuracies'] else None,
            'final_val_acc': self.metrics['val_accuracies'][-1] if self.metrics['val_accuracies'] else None,
            'epochs_completed': len(self.metrics['train_losses'])
        }


# ModelTrainer Class
Manages the training process for PyTorch models.

This class handles the entire training lifecycle including:
- Running training and validation epochs
- Computing and tracking metrics
- Implementing early stopping
- Managing the learning rate scheduler
- Supporting Optuna trials for hyperparameter optimization

The class is designed to work with any PyTorch model, loss function, and optimizer,
making it highly reusable across different projects.



In [ ]:
!pip install optuna
!pip install optuna_dashboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.9 MB/s eta 0:00:00


In [ ]:
import optuna
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from typing import Optional, Tuple, List
import copy

@dataclass
class SchedulerConfig:
    """Simple configuration class for learning rate scheduler"""
    mode: str = 'min'
    factor: float = 0.1
    patience: int = 5

    def create_scheduler(self, optimizer):
        """Create a ReduceLROnPlateau scheduler with the specified configuration"""
        return torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=self.mode,
            factor=self.factor,
            patience=self.patience
        )

class ModelTrainer:
    def __init__(self, model, criterion, optimizer,
                 scheduler_config=None, device=None, patience=10):
        """
        Initialize the model trainer

        Args:
            model: PyTorch model to train
            criterion: Loss function
            optimizer: Optimization algorithm
            scheduler_config: Configuration for learning rate scheduler (optional)
            device: Device to run training on ('cuda' or 'cpu')
            patience: Number of epochs to wait before early stopping
        """
        # Set device if not provided
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize components
        self.model = model.to(self.device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler_config.create_scheduler(optimizer) if scheduler_config else None
        self.patience = patience
        self.metrics = TrainingMetrics()

        print(f"ModelTrainer initialized with device: {self.device}")

    def train_epoch(self, train_loader):
        """
        Train for a single epoch

        Args:
            train_loader: DataLoader for training data

        Returns:
            tuple: (avg_loss, accuracy, predictions, targets)
        """
        self.model.train()  # Set model to training mode
        total_loss = 0
        correct = 0
        total = 0
        predictions = []
        targets = []

        # Iterate through batches
        for X_batch, y_batch in train_loader:
            # Move data to device
            X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

            # Zero gradients
            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(X_batch)
            loss = self.criterion(outputs, y_batch)

            # Backward pass and optimize
            loss.backward()
            self.optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
            targets.extend(y_batch.cpu().numpy())
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

            # Accumulate loss
            total_loss += loss.item()

        # Calculate average loss and accuracy
        avg_loss = total_loss / len(train_loader)
        accuracy = 100 * correct / total if total > 0 else 0

        return avg_loss, accuracy, predictions, targets

    def validate(self, val_loader):
        """
        Validate the model

        Args:
            val_loader: DataLoader for validation data

        Returns:
            tuple: (avg_loss, accuracy, predictions, targets)
        """
        self.model.eval()  # Set model to evaluation mode
        total_loss = 0
        correct = 0
        total = 0
        predictions = []
        targets = []

        # No gradient calculation during validation
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Move data to device
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

                # Forward pass
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                predictions.extend(predicted.cpu().numpy())
                targets.extend(y_batch.cpu().numpy())
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

                # Accumulate loss
                total_loss += loss.item()

        # Calculate average loss and accuracy
        avg_loss = total_loss / len(val_loader)
        accuracy = 100 * correct / total if total > 0 else 0

        return avg_loss, accuracy, predictions, targets

    def train(self, train_loader, val_loader, num_epochs, trial=None):
        """
        Complete training process with early stopping

        Args:
            train_loader: DataLoader for training data
            val_loader: DataLoader for validation data
            num_epochs: Maximum number of epochs to train
            trial: Optuna trial for hyperparameter optimization (optional)

        Returns:
            TrainingMetrics: Object containing training history
        """
        patience_counter = 0
        best_model_state = None

        print(f"Starting training for {num_epochs} epochs")
        for epoch in range(num_epochs):
            # Training phase
            train_loss, train_acc, pred_train, target_train = self.train_epoch(train_loader)

            # Validation phase
            val_loss, val_acc, pred_val, target_val = self.validate(val_loader)

            # Update metrics and check if best model
            is_best = self.metrics.update(train_loss, train_acc, val_loss, val_acc, pred_val, target_val)

            # Print progress
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

            # Learning rate scheduling
            if self.scheduler is not None:
                old_lr = self.optimizer.param_groups[0]['lr']
                self.scheduler.step(val_loss)
                new_lr = self.optimizer.param_groups[0]['lr']
                if new_lr != old_lr:
                    print(f"Learning rate adjusted: {old_lr:.6f} -> {new_lr:.6f}")

            # Optuna pruning
            if trial:
                trial.report(val_loss, epoch)
                if trial.should_prune():
                    print("Trial pruned by Optuna")
                    raise optuna.TrialPruned()

            # Early stopping logic
            if is_best:
                best_model_state = copy.deepcopy(self.model.state_dict())
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break

        # Store final data for confusion matrix
        self.metrics.update_confusion_matrix_data(pred_train, target_train, pred_val, target_val)

        # Restore best model
        if best_model_state:
            self.model.load_state_dict(best_model_state)
            self.metrics.best_model_state = best_model_state
            print("Restored best model from training")

        # Print training summary
        summary = self.metrics.get_summary()
        print(f"Best validation loss: {summary['best_val_loss']:.4f}")

        return self.metrics


# OptunaTrainer Class
Manages hyperparameter optimization using Optuna.

This class encapsulates the hyperparameter optimization process, including:
- Defining the hyperparameter search space
- Creating models with different hyperparameter configurations
- Training and evaluating each model configuration
- Optimizing to find the best parameters
- Visualizing the optimization process with Optuna Dashboard

The class is designed to work with any model class that accepts the defined
hyperparameters and follows a consistent interface.


In [ ]:
from optuna.trial import Trial
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from typing import Type, Dict, Tuple
import threading
from datetime import datetime
try:
    from google.colab import output
    from optuna_dashboard import run_server
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

class OptunaTrainer:
    def __init__(
        self,
        model_class,
        train_loader,
        val_loader,
        criterion,
        hyperparameter_ranges=None,
        n_trials=100,
        n_jobs=2,
        study_name=None
    ):
        """
        Initialize the Optuna trainer

        Args:
            model_class: The model class to optimize
            train_loader: DataLoader for training data
            val_loader: DataLoader for validation data
            criterion: Loss function
            hyperparameter_ranges: Dictionary defining the search space (optional)
            n_trials: Number of trials to run
            n_jobs: Number of parallel jobs
            study_name: Name for the Optuna study
        """
        # Initialize components
        self.model_class = model_class
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.n_trials = n_trials
        self.n_jobs = n_jobs
        self.study_name = study_name or f"optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        # Set default hyperparameter search space if not provided
        self.hyperparameter_ranges = hyperparameter_ranges or {
            'learning_rate': {
                'type': 'float',
                'low': 1e-5,
                'high': 1e-1,
                'log': True
            },
            'dropout_rate1': {
                'type': 'float',
                'low': 0.1,
                'high': 0.7
            },
            'dropout_rate2': {
                'type': 'float',
                'low': 0.1,
                'high': 0.7
            }
        }

        # Initial configuration for first trial
        self.initial_config = {
            'learning_rate': 0.02,
            'dropout_rate1': 0.2,
            'dropout_rate2': 0.5
        }

        print(f"OptunaTrainer initialized with {n_trials} trials and {n_jobs} parallel jobs")

    def create_model(self, trial):
        """
        Create model and optimizer with trial parameters

        Args:
            trial: Optuna trial object

        Returns:
            tuple: (model, optimizer)
        """
        # Create model with trial parameters
        model = self.model_class(
            dropout_rate1=trial.params['dropout_rate1'],
            dropout_rate2=trial.params['dropout_rate2'],
        )

        # Create optimizer
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=trial.params['learning_rate'],
        )

        return model, optimizer

    def objective(self, trial):
        """
        Objective function for Optuna optimization

        Args:
            trial: Optuna trial object

        Returns:
            float: Validation loss to minimize
        """
        print(f"Starting trial {trial.number}")

        # Define hyperparameters for this trial
        for param_name, param_config in self.hyperparameter_ranges.items():
            if param_config['type'] == 'float':
                trial.suggest_float(
                    param_name,
                    param_config['low'],
                    param_config['high'],
                    log=param_config.get('log', False)
                )
            elif param_config['type'] == 'int':
                trial.suggest_int(
                    param_name,
                    param_config['low'],
                    param_config['high'],
                    step=param_config.get('step', 1),
                    log=param_config.get('log', False)
                )
            elif param_config['type'] == 'categorical':
                trial.suggest_categorical(param_name, param_config['choices'])

        # Create model and optimizer
        model, optimizer = self.create_model(trial)

        # Create scheduler
        scheduler_config = SchedulerConfig()

        # Create trainer and run training
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        trainer = ModelTrainer(
            model=model,
            criterion=self.criterion,
            optimizer=optimizer,
            scheduler_config=scheduler_config,
            device=device
        )

        # Train model
        try:
            metrics = trainer.train(
                train_loader=self.train_loader,
                val_loader=self.val_loader,
                num_epochs=20,
                trial=trial
            )

            print(f"Trial {trial.number} finished with loss: {metrics.best_val_loss:.4f}")
            return metrics.best_val_loss

        except optuna.TrialPruned:
            print(f"Trial {trial.number} pruned")
            raise

    def run(self):
        """
        Run the hyperparameter optimization

        Returns:
            optuna.Study: Study object with optimization results
        """
        print(f"Starting optimization with {self.n_trials} trials")

        # Create study
        db_path = "sqlite:///optuna.db"
        study = optuna.create_study(
            direction='minimize',
            pruner=optuna.pruners.MedianPruner(),
            sampler=optuna.samplers.TPESampler(seed=42),
            storage=db_path,
            study_name=self.study_name,
            load_if_exists=True
        )

        # Start with initial configuration
        study.enqueue_trial(self.initial_config)

        # Run optimization
        study.optimize(self.objective, n_trials=self.n_trials, n_jobs=self.n_jobs)

        # Log results
        print("Optimization completed")
        print(f"Best trial: {study.best_trial.number}")
        print(f"Best value: {study.best_value}")
        print(f"Best parameters: {study.best_params}")

        return study

    def start_dashboard(self, port=8082):
        """
        Start Optuna Dashboard in a separate thread

        Args:
            port: Port number for the dashboard
        """
        if not COLAB_ENV:
            print("Optuna Dashboard requires Google Colab environment")
            return

        thread = threading.Thread(
            target=run_server,
            args=("sqlite:///optuna.db",),
            kwargs={"port": port, "access_log_format": ""}
        )
        thread.daemon = True  # Daemon threads exit when main thread exits
        thread.start()
        output.serve_kernel_port_as_iframe(port, path='/dashboard/')
        print(f"Optuna Dashboard started on port {port}")


# ModelTrainingPipeline Class
Orchestrates the entire model training workflow.

This class coordinates the end-to-end machine learning pipeline:
- Data preparation and splitting
- Hyperparameter optimization
- Final model training with best parameters
- Model evaluation and saving

It provides a high-level interface that abstracts away the complexity of the
individual components while maintaining flexibility for customization.


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import os
from datetime import datetime
import optuna
from typing import Dict, Tuple, Type, Optional

class ModelTrainingPipeline:
    def __init__(self, model_class, X, y, criterion=None, seed=42):
        """
        Initialize the model training pipeline

        Args:
            model_class: The model class to train
            X: Input features
            y: Target labels
            criterion: Loss function (default: CrossEntropyLoss)
            seed: Random seed for reproducibility
        """
        # Set random seeds for reproducibility
        torch.manual_seed(seed)
        np.random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

        # Initialize components
        self.model_class = model_class
        self.X = X
        self.y = y
        self.criterion = criterion or nn.CrossEntropyLoss()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize results storage
        self.best_params = None
        self.final_model = None
        self.final_metrics = None
        self.final_data = None
        self.study = None

        print("ModelTrainingPipeline initialized")
        print(f"Device: {self.device}")
        print(f"Input shape: {X.shape}")

    def prepare_data(self):
        """
        Prepare data for training using the existing preprocessing function

        Returns:
            tuple: (train_loader, val_loader, train_val_loader, test_loader, scaler)
        """
        print("Preparing data")
        # Use your existing preprocessing function
        train_loader, val_loader, train_val_loader, test_loader, scaler = preprocess_data_subsample_three_splits(
            self.X, self.y
        )

        print(f"Data prepared: {len(train_loader)} training batches, {len(val_loader)} validation batches")
        return train_loader, val_loader, train_val_loader, test_loader, scaler

    def run_hyperparameter_optimization(self, n_trials=50, n_jobs=2, custom_ranges=None):
        """
        Run hyperparameter optimization

        Args:
            n_trials: Number of trials to run
            n_jobs: Number of parallel jobs
            custom_ranges: Custom hyperparameter search space

        Returns:
            optuna.Study: Study object with optimization results
        """
        print(f"Starting hyperparameter optimization with {n_trials} trials")

        # Prepare data
        train_loader, val_loader, _, _, _ = self.prepare_data()

        # Create Optuna trainer
        optuna_trainer = OptunaTrainer(
            model_class=self.model_class,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=self.criterion,
            hyperparameter_ranges=custom_ranges,
            n_trials=n_trials,
            n_jobs=n_jobs,
            study_name=f"optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        )

        # Start dashboard (only works in Colab)
        optuna_trainer.start_dashboard()

        # Run optimization
        self.study = optuna_trainer.run()

        # Store best parameters
        self.best_params = self.study.best_params
        print(f"Best parameters: {self.best_params}")
        print(f"Best validation loss: {self.study.best_value:.4f}")

        return self.study

    def train_final_model(self, num_epochs=50, custom_params=None):
        """
        Train final model with best parameters

        Args:
            num_epochs: Number of epochs to train
            custom_params: Custom parameters to use instead of best params

        Returns:
            tuple: (model, metrics)
        """
        # Ensure we have parameters to use
        params_to_use = custom_params or self.best_params
        if params_to_use is None:
            raise ValueError("No parameters available. Run hyperparameter optimization first or provide custom_params.")

        print(f"Training final model for {num_epochs} epochs with parameters: {params_to_use}")

        # Prepare data - using train_val_loader for final training
        _, _, train_val_loader, test_loader, _ = self.prepare_data()

        # Initialize model with parameters
        model = self.model_class(
            dropout_rate1=params_to_use['dropout_rate1'],
            dropout_rate2=params_to_use['dropout_rate2']
        ).to(self.device)

        # Initialize optimizer
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=params_to_use['learning_rate']
        )

        # Initialize scheduler
        scheduler_config = SchedulerConfig()

        # Create trainer
        trainer = ModelTrainer(
            model=model,
            criterion=self.criterion,
            optimizer=optimizer,
            scheduler_config=scheduler_config,
            device=self.device
        )

        # Train model
        training_results = trainer.train(
            train_loader=train_val_loader,
            val_loader=test_loader,
            num_epochs=num_epochs
        )

        # Store results
        self.final_model = model
        self.final_metrics = training_results.metrics
        self.final_data = training_results.data

        # Evaluate on test set
        test_loss, test_acc, test_preds, test_targets = trainer.validate(test_loader)
        print(f"Final test loss: {test_loss:.4f}")
        print(f"Final test accuracy: {test_acc:.2f}%")

        # Return results
        final_results = {
            'model': model,
            'metrics': training_results.metrics,
            'data': training_results.data,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'test_preds': test_preds,
            'test_targets': test_targets,
            'params': params_to_use
        }

        return model, final_results

    def save_model(self, path="./saved_model.pt"):
        """
        Save the trained model to disk

        Args:
            path: Path to save the model
        """
        if self.final_model is None:
            raise ValueError("No model to save. Train a model first.")

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(path)), exist_ok=True)

        # Prepare model info
        model_info = {
            'model_state_dict': self.final_model.state_dict(),
            'parameters': self.best_params or {},
            'metrics': {k: v[-1] for k, v in self.final_metrics.items()} if self.final_metrics else {},
            'saved_at': datetime.now().isoformat()
        }

        # Save model
        torch.save(model_info, path)
        print(f"Model saved to {path}")

    def load_model(self, path="./saved_model.pt"):
        """
        Load a saved model

        Args:
            path: Path to the saved model

        Returns:
            model: Loaded model
        """
        # Load model info
        model_info = torch.load(path, map_location=self.device)

        # Extract parameters
        params = model_info.get('parameters', {})

        # Create model
        model = self.model_class(
            dropout_rate1=params.get('dropout_rate1', 0.2),
            dropout_rate2=params.get('dropout_rate2', 0.5)
        ).to(self.device)

        # Load model weights
        model.load_state_dict(model_info['model_state_dict'])

        # Update model and parameters
        self.final_model = model
        self.best_params = params

        print(f"Model loaded from {path}")
        print(f"Parameters: {params}")

        return model


def run_complete_pipeline(X, y, model_class, n_trials=50, final_epochs=50):
    """
    Run the complete model training pipeline

    Args:
        X: Input features
        y: Target labels
        model_class: Model class to train
        n_trials: Number of hyperparameter optimization trials
        final_epochs: Number of epochs for final model training

    Returns:
        pipeline: Trained pipeline object
    """
    # Initialize the pipeline
    pipeline = ModelTrainingPipeline(model_class, X, y)

    # Run hyperparameter optimization
    study = pipeline.run_hyperparameter_optimization(n_trials=n_trials)

    # Train final model with best parameters
    final_model, final_results = pipeline.train_final_model(num_epochs=final_epochs)

    # Save the model
    pipeline.save_model("./best_model.pt")

    return pipeline

In [ ]:
# Initialize the pipeline
pipeline = ModelTrainingPipeline(PVFaultClassifier, X, y)


ModelTrainingPipeline initialized
Device: cpu
Input shape: (1373798, 6)


In [ ]:
# Run hyperparameter optimization
study = pipeline.run_hyperparameter_optimization(n_trials=20)


Starting hyperparameter optimization with 20 trials
Preparing data
Data prepared: 657 training batches, 141 validation batches
OptunaTrainer initialized with 20 trials and 2 parallel jobs


Exception in thread Thread-9 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner


<IPython.core.display.Javascript object>

    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
TypeError: run_server() got an unexpected keyword argument 'access_log_format'


Optuna Dashboard started on port 8082
Starting optimization with 20 trials


...

Epoch [20/20]
Train Loss: 0.0302, Train Acc: 99.03%
Val Loss: 0.0308, Val Acc: 99.02%
Restored best model from training
Best validation loss: 0.0308
Trial 17 finished with loss: 0.0308
Starting trial 19
ModelTrainer initialized with device: cpu
Starting training for 20 epochs
Epoch [12/20]
Train Loss: 0.0648, Train Acc: 98.00%
Val Loss: 0.0689, Val Acc: 98.07%
Learning rate adjusted: 0.009623 -> 0.000962
Epoch [1/20]
Train Loss: 0.2080, Train Acc: 91.80%
Val Loss: 0.0847, Val Acc: 97.49%
Epoch [13/20]
Train Loss: 0.0480, Train Acc: 98.53%
Val Loss: 0.0374, Val Acc: 98.89%
Epoch [2/20]
Train Loss: 0.0864, Train Acc: 97.25%
Val Loss: 0.0835, Val Acc: 97.56%
Epoch [14/20]
Train Loss: 0.0405, Train Acc: 98.70%
Val Loss: 0.0361, Val Acc: 98.69%
Epoch [3/20]
Train Loss: 0.0806, Train Acc: 97.36%
Val Loss: 0.0832, Val Acc: 97.51%
Epoch [15/20]
Train Loss: 0.0398, Train Acc: 98.71%
Val Loss: 0.0339, Val Acc: 98.84%
Epoch [4/20]
Train Loss: 0.0746, Train Acc: 97.49%
Val Loss: 0.0933, Val Acc: 9

[I 2025-02-25 23:06:02,882] Trial 18 finished with value: 0.032427241211444975 and parameters: {'learning_rate': 0.009623066122972089, 'dropout_rate1': 0.20343056666888465, 'dropout_rate2': 0.2321409044901617}. Best is trial 7 with value: 0.028208779666183934.


Epoch [20/20]
Train Loss: 0.0369, Train Acc: 98.79%
Val Loss: 0.0324, Val Acc: 98.98%
Restored best model from training
Best validation loss: 0.0324
Trial 18 finished with loss: 0.0324
Epoch [9/20]
Train Loss: 0.0584, Train Acc: 97.99%
Val Loss: 0.0587, Val Acc: 98.13%
Epoch [10/20]
Train Loss: 0.0605, Train Acc: 98.03%
Val Loss: 0.0785, Val Acc: 96.67%


[I 2025-02-25 23:06:11,013] Trial 19 pruned. 


Epoch [11/20]
Train Loss: 0.0556, Train Acc: 98.20%
Val Loss: 0.0544, Val Acc: 98.27%
Trial pruned by Optuna
Trial 19 pruned
Optimization completed
Best trial: 7
Best value: 0.028208779666183934
Best parameters: {'learning_rate': 0.0018988577680075385, 'dropout_rate1': 0.12369830238514777, 'dropout_rate2': 0.17021212515398046}
Best parameters: {'learning_rate': 0.0018988577680075385, 'dropout_rate1': 0.12369830238514777, 'dropout_rate2': 0.17021212515398046}
Best validation loss: 0.0282


In [ ]:
# Train final model with best parameters
final_model, results = pipeline.train_final_model(num_epochs=30)

Training final model for 30 epochs with parameters: {'learning_rate': 0.0018988577680075385, 'dropout_rate1': 0.12369830238514777, 'dropout_rate2': 0.17021212515398046}
Preparing data
Data prepared: 657 training batches, 141 validation batches
ModelTrainer initialized with device: cpu
Starting training for 30 epochs
Epoch [1/30]
Train Loss: 0.3109, Train Acc: 88.48%
Val Loss: 0.1020, Val Acc: 96.62%
Epoch [2/30]
Train Loss: 0.0933, Train Acc: 96.92%
Val Loss: 0.0618, Val Acc: 97.93%
Epoch [3/30]
Train Loss: 0.0710, Train Acc: 97.66%
Val Loss: 0.0483, Val Acc: 98.42%
Epoch [4/30]
Train Loss: 0.0620, Train Acc: 97.85%
Val Loss: 0.0480, Val Acc: 98.36%
Epoch [5/30]
Train Loss: 0.0565, Train Acc: 98.11%
Val Loss: 0.0351, Val Acc: 98.87%
Epoch [6/30]
Train Loss: 0.0517, Train Acc: 98.20%
Val Loss: 0.0397, Val Acc: 98.89%
Epoch [7/30]
Train Loss: 0.0492, Train Acc: 98.33%
Val Loss: 0.0360, Val Acc: 98.67%
Epoch [8/30]
Train Loss: 0.0447, Train Acc: 98.43%
Val Loss: 0.0393, Val Acc: 98.91%
Ep